In [53]:
import boto3

# 建立 DynamoDB 資源
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

# 創建表格
table = dynamodb.create_table(
    TableName='boto3_test',
    KeySchema=[
        {'AttributeName': 'UserID', 'KeyType': 'HASH'},  # Partition Key
        {'AttributeName': 'Attempt', 'KeyType': 'RANGE'}  # Sort Key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'UserID', 'AttributeType': 'S'},  # String
        {'AttributeName': 'Attempt', 'AttributeType': 'N'},  # Number
        {'AttributeName': 'StudentID', 'AttributeType': 'S'},  # 供 GSI 使用
        {'AttributeName': 'Email', 'AttributeType': 'S'}  # 供 GSI 使用
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    },
    GlobalSecondaryIndexes=[
        {
            'IndexName': 'StudentIDIndex',
            'KeySchema': [
                {'AttributeName': 'StudentID', 'KeyType': 'HASH'}
            ],
            'Projection': {'ProjectionType': 'ALL'},
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 2,
                'WriteCapacityUnits': 2
            }
        },
        {
            'IndexName': 'EmailIndex',
            'KeySchema': [
                {'AttributeName': 'Email', 'KeyType': 'HASH'}
            ],
            'Projection': {'ProjectionType': 'ALL'},
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 2,
                'WriteCapacityUnits': 2
            }
        }
    ]
)

print("正在創建表格...")
table.wait_until_exists()
print("表格創建成功！")

正在創建表格...
表格創建成功！


In [4]:
table = dynamodb.Table('boto3_test')

for i in range(10):
    table.put_item(
    Item={
        'UserID': 'jlai23',
        'Attempt': i,
        'StudentID': 'B10601043',
        'Email': 'b10601043@g.ntu.edu.tw'
    }
)

print("數據插入成功！")

數據插入成功！


In [5]:
table = dynamodb.Table('boto3_test')

for i in range(100):
    table.put_item(
    Item={
        'UserID': 'jlai23_ad',
        'Attempt': i,
        'StudentID': 'B10601043',
        'Email': 'b10601043@g.ntu.edu.tw'
    }
)

print("數據插入成功！")

數據插入成功！


In [ ]:
import boto3
from boto3.dynamodb.conditions import Key

# 建立 DynamoDB 資源
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

table = dynamodb.Table('boto3_test')

response = table.query(
    KeyConditionExpression=Key('UserID').eq('jlai23'),# 取得某一Partition key的table
    ProjectionExpression='Attempt , UserID'# 想取出的欄位
)

print(response['Items'])
print(type(response['Items']))


[{'UserID': 'jlai23', 'Attempt': Decimal('0')}, {'UserID': 'jlai23', 'Attempt': Decimal('1')}, {'UserID': 'jlai23', 'Attempt': Decimal('2')}, {'UserID': 'jlai23', 'Attempt': Decimal('3')}, {'UserID': 'jlai23', 'Attempt': Decimal('4')}, {'UserID': 'jlai23', 'Attempt': Decimal('5')}, {'UserID': 'jlai23', 'Attempt': Decimal('6')}, {'UserID': 'jlai23', 'Attempt': Decimal('7')}, {'UserID': 'jlai23', 'Attempt': Decimal('8')}, {'UserID': 'jlai23', 'Attempt': Decimal('9')}]
<class 'list'>


In [10]:
subtable = response['Items']

print(subtable[0])

{'UserID': 'jlai23', 'Attempt': Decimal('0')}


In [11]:
import pandas as pd

df = pd.DataFrame(subtable) #　將取得的列表轉化成dataframe

print(df)

   UserID Attempt
0  jlai23       0
1  jlai23       1
2  jlai23       2
3  jlai23       3
4  jlai23       4
5  jlai23       5
6  jlai23       6
7  jlai23       7
8  jlai23       8
9  jlai23       9


In [16]:
response = table.scan()

whole_table = response['Items']

df = pd.DataFrame(whole_table)

print(df)

        UserID Attempt                   Email  StudentID
0       jlai23       0  b10601043@g.ntu.edu.tw  B10601043
1       jlai23       1  b10601043@g.ntu.edu.tw  B10601043
2       jlai23       2  b10601043@g.ntu.edu.tw  B10601043
3       jlai23       3  b10601043@g.ntu.edu.tw  B10601043
4       jlai23       4  b10601043@g.ntu.edu.tw  B10601043
..         ...     ...                     ...        ...
105  jlai23_ad      95  b10601043@g.ntu.edu.tw  B10601043
106  jlai23_ad      96  b10601043@g.ntu.edu.tw  B10601043
107  jlai23_ad      97  b10601043@g.ntu.edu.tw  B10601043
108  jlai23_ad      98  b10601043@g.ntu.edu.tw  B10601043
109  jlai23_ad      99  b10601043@g.ntu.edu.tw  B10601043

[110 rows x 4 columns]


In [18]:
from boto3.dynamodb.conditions import Attr

table = dynamodb.Table('boto3_test')

response = table.scan(
    FilterExpression=Attr('Attempt').gt(80)  # 過濾 Attempt > 80
)

items_to_delete = response['Items']

df_to_delete = pd.DataFrame(items_to_delete)

print(df_to_delete)

       UserID Attempt                   Email  StudentID
0   jlai23_ad      81  b10601043@g.ntu.edu.tw  B10601043
1   jlai23_ad      82  b10601043@g.ntu.edu.tw  B10601043
2   jlai23_ad      83  b10601043@g.ntu.edu.tw  B10601043
3   jlai23_ad      84  b10601043@g.ntu.edu.tw  B10601043
4   jlai23_ad      85  b10601043@g.ntu.edu.tw  B10601043
5   jlai23_ad      86  b10601043@g.ntu.edu.tw  B10601043
6   jlai23_ad      87  b10601043@g.ntu.edu.tw  B10601043
7   jlai23_ad      88  b10601043@g.ntu.edu.tw  B10601043
8   jlai23_ad      89  b10601043@g.ntu.edu.tw  B10601043
9   jlai23_ad      90  b10601043@g.ntu.edu.tw  B10601043
10  jlai23_ad      91  b10601043@g.ntu.edu.tw  B10601043
11  jlai23_ad      92  b10601043@g.ntu.edu.tw  B10601043
12  jlai23_ad      93  b10601043@g.ntu.edu.tw  B10601043
13  jlai23_ad      94  b10601043@g.ntu.edu.tw  B10601043
14  jlai23_ad      95  b10601043@g.ntu.edu.tw  B10601043
15  jlai23_ad      96  b10601043@g.ntu.edu.tw  B10601043
16  jlai23_ad      97  b1060104

In [20]:
table = dynamodb.Table('boto3_test')

for item in items_to_delete:
    
    table.delete_item(
        Key = {
            'UserID': item['UserID'],  # Partition Key
            'Attempt': item['Attempt']  # Sort Key
        }
    )

response = table.scan()

subtable = response['Items']

print(pd.DataFrame(subtable))

       UserID Attempt                   Email  StudentID
0      jlai23       0  b10601043@g.ntu.edu.tw  B10601043
1      jlai23       1  b10601043@g.ntu.edu.tw  B10601043
2      jlai23       2  b10601043@g.ntu.edu.tw  B10601043
3      jlai23       3  b10601043@g.ntu.edu.tw  B10601043
4      jlai23       4  b10601043@g.ntu.edu.tw  B10601043
..        ...     ...                     ...        ...
86  jlai23_ad      76  b10601043@g.ntu.edu.tw  B10601043
87  jlai23_ad      77  b10601043@g.ntu.edu.tw  B10601043
88  jlai23_ad      78  b10601043@g.ntu.edu.tw  B10601043
89  jlai23_ad      79  b10601043@g.ntu.edu.tw  B10601043
90  jlai23_ad      80  b10601043@g.ntu.edu.tw  B10601043

[91 rows x 4 columns]


In [21]:
# 刪除表格
table.delete()

{'TableDescription': {'TableName': 'boto3_test',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:159761224094:table/boto3_test',
  'TableId': '3533c168-a0dd-4c9b-bf43-076e7885c74e',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'J50GCMNEVP2JMH059TSUIN6P43VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 22 Mar 2025 08:43:39 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '356',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'J50GCMNEVP2JMH059TSUIN6P43VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3708051704'},
  'RetryAttempts': 0}}

In [49]:
import pandas as pd

# 讀取 CSV 檔案
df = pd.read_csv("rice_growth_template.csv")

column_names = df.columns.tolist()

print(type(column_names[1]))

<class 'str'>


In [121]:
import pandas as pd
import boto3

# 讀取 CSV 檔案
df = pd.read_csv("rice_growth_template.csv")

column_names = df.columns.tolist()

# 建立 DynamoDB 資源
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

# 假設 Partition_Key 和 Sort_Key 都是字串類型
Partition_Key = column_names[0]  # 第一欄作為 Partition Key
Sort_Key = column_names[1]       # 第二欄作為 Sort Key

# 創建表格
table = dynamodb.create_table(
    TableName='rice_growth_template',
    KeySchema=[
        {'AttributeName': Partition_Key, 'KeyType': 'HASH'},  # Partition Key (variety)
        {'AttributeName': Sort_Key, 'KeyType': 'RANGE'}  # Sort Key (number)
    ],
    AttributeDefinitions=[
        {'AttributeName': Partition_Key, 'AttributeType': 'S'},  # Partition Key
        {'AttributeName': Sort_Key, 'AttributeType': 'N'},       # Sort Key
        {'AttributeName': column_names[2], 'AttributeType': 'S'},  # 其他欄位
        {'AttributeName': column_names[3], 'AttributeType': 'S'},
        {'AttributeName': column_names[4], 'AttributeType': 'S'},
        
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    },
    GlobalSecondaryIndexes=[
        # 設定每個欄位作為二級索引
        {
            'IndexName': f"{column_names[2]}_Index",  # Index name for record_time
            'KeySchema': [
                {'AttributeName': column_names[2], 'KeyType': 'HASH'},  # Partition Key for GSI
                {'AttributeName': Sort_Key, 'KeyType': 'RANGE'}  # Sort Key for GSI
            ],
            'Projection': {
                'ProjectionType': 'ALL'  # 返回所有欄位
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        },
        {
            'IndexName': f"{column_names[3]}_Index",  # Index name for height
            'KeySchema': [
                {'AttributeName': column_names[3], 'KeyType': 'HASH'},  # Partition Key for GSI
                {'AttributeName': Sort_Key, 'KeyType': 'RANGE'}  # Sort Key for GSI
            ],
            'Projection': {
                'ProjectionType': 'ALL'  # 返回所有欄位
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        },
        {
            'IndexName': f"{column_names[4]}_Index",  # Index name for fertility_cycle
            'KeySchema': [
                {'AttributeName': column_names[4], 'KeyType': 'HASH'},  # Partition Key for GSI
                {'AttributeName': Sort_Key, 'KeyType': 'RANGE'}  # Sort Key for GSI
            ],
            'Projection': {
                'ProjectionType': 'ALL'  # 返回所有欄位
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        },
        {
            'IndexName': f"{column_names[5]}_Index",  # Index name for fertility_cycle
            'KeySchema': [
                {'AttributeName': column_names[4], 'KeyType': 'HASH'},  # Partition Key for GSI
                {'AttributeName': Sort_Key, 'KeyType': 'RANGE'}  # Sort Key for GSI
            ],
            'Projection': {
                'ProjectionType': 'ALL'  # 返回所有欄位
            },
            'ProvisionedThroughput': {
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            }
        }
    ]
)

print("正在創建表格...")
table.wait_until_exists()
print("表格創建成功！")


正在創建表格...
表格創建成功！


In [122]:
row = df.iloc[70]
print(column_names)
print(type(row[column_names[3]]))
print(len(df))
print(df)

['number', 'record_time', 'height', 'fertility_cycle', 'tillering', 'variety']
<class 'numpy.int64'>
120
     number  record_time  height  fertility_cycle  tillering   variety
0         1            0    15.0                5          0  Taoyuan3
1         1            2    36.0                8          3  Taoyuan3
2         1            4    50.0                9          4  Taoyuan3
3         1            6    60.0               11          8  Taoyuan3
4         1            8    66.0               13         13  Taoyuan3
..      ...          ...     ...              ...        ...       ...
115      10            2    16.0                5          1  Tainan11
116      10            4    34.0                7          4  Tainan11
117      10            6    60.0                9          6  Tainan11
118      10            8    72.0               11          8  Tainan11
119      10           10    92.0               12          9  Tainan11

[120 rows x 6 columns]


In [124]:
table = dynamodb.Table('rice_growth_template')

for i in range(len(df)):
    row = df.iloc[i]
    table.put_item(    
        Item={
            column_names[0]: str(row[column_names[0]]),
            column_names[1]: int(row[column_names[1]]),
            column_names[2]: str(row[column_names[2]]),
            column_names[3]: str(row[column_names[3]]),
            column_names[4]: str(row[column_names[4]]),
            column_names[5]: str(row[column_names[5]])
    })


print("物件插入成功")



物件插入成功
